In [1]:
import glob2
import pandas as pd
import numpy as np
import json
from natsort import natsorted
from datetime import datetime

In [2]:
dataset_path = '/home/ubuntu/SoccerNet-code/data/data/europe_uefa-champions-league'
dataframe_path = '../data/SoccerNet/finishings_dataset.pkl'

In [3]:
def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

In [4]:
def check_inference(json_data, game_df, threshold=0.5):        
    result = []
    for time_annotated in game_df['Tempo (segundos)']:
        status = False
        for inference in json_data['inferences']:
            if float(inference['probability'])>=threshold:
                start = datetime.strptime(inference['start_time'], '%H:%M:%S')
                start = (start.hour * 60 + start.minute) * 60 + start.second
                end = datetime.strptime(inference['end_time'], '%H:%M:%S')
                end = (end.hour * 60 + end.minute) * 60 + end.second
                status = time_in_range(start, end, time_annotated)
                if status == True:
                    break
        result.append(status)
    return result

In [5]:
def eval_metric(dataset_path, dataframe_path, threshold=0.5):
    json_paths = natsorted(glob2.glob(dataset_path + '/**/*.json'))
    dataset_df = pd.read_pickle(dataframe_path)
    
    results = []
    flat_list = []
    total_inferences = 0
    total_annotations = 0

    for json_path in json_paths:
        jsonFile = open(json_path, "r")
        json_data = json.load(jsonFile)
        jsonFile.close()
        game = json_data['video']['path'].split('/')[-2]
        period = [int(c) for c in json_data['video']['path'].split('/')[-1] if c.isdigit()][0]
        game_df = dataset_df[(dataset_df['Diretorio (jogo)'] == game) & (np.array(dataset_df['Periodo'], dtype=int)==period)]
        total_inferences += len(json_data['inferences'])
        total_annotations += len(game_df)
        game_results = check_inference(json_data, game_df, threshold=threshold)
        results.append(game_results)
        print(game, game_results, period)
        
    for sublist in results:
        for item in sublist:
            flat_list.append(item)

    _, (fp, tp) = np.unique(flat_list, return_counts=True)
    precision = tp/(fp + tp)
    return precision

In [6]:
#[False, False, False, False, False, False, True, False, False, False, True, False, True]
#[False, False, False, False, False, False, True, False, False, False, True, False, True]
#[True, False, False, False, False, False, True, False, False, False, True, False, True]

In [7]:
eval_metric(dataset_path, dataframe_path, threshold=0.5)

2015-09-15 - 21-45 Galatasaray 0 - 2 Atl. Madrid [True, False, False, False, False, False, True, False, False, False, True, False, True] 1
2015-09-15 - 21-45 Galatasaray 0 - 2 Atl. Madrid [False, False, True, False, True, False, False, False, False, True] 2
2015-09-15 - 21-45 Manchester City 1 - 2 Juventus [True, True, True, True, True, True, True, True, True] 1
2015-09-15 - 21-45 Manchester City 1 - 2 Juventus [True, True, True, True, True, False, True, True, True] 2
2015-09-15 - 21-45 PSV 2 - 1 Manchester United [True, True, True, True, True, False] 1
2015-09-15 - 21-45 PSV 2 - 1 Manchester United [True, True, True, True, True, True, True, True, True] 2
2015-09-15 - 21-45 Paris SG 2 - 0 Malmo FF [False, True, False, True, True, True, True, False] 1
2015-09-15 - 21-45 Paris SG 2 - 0 Malmo FF [False, False, True, True, True, True, False, False, True, False, False] 2
2015-09-15 - 21-45 Real Madrid 4 - 0 Shakhtar Donetsk [False, False, False, True, True, True, True, False, True, True, Fa

2015-11-24 - 22-45 Barcelona 6 - 1 AS Roma [True, True, True, True] 2
2015-11-24 - 22-45 Bayern Munich 4 - 0 Olympiakos Piraeus [True, True, False] 1
2015-11-24 - 22-45 Bayern Munich 4 - 0 Olympiakos Piraeus [True] 2
2015-11-24 - 22-45 FC Porto 0 - 2 Dyn. Kiev [True] 1
2015-11-24 - 22-45 FC Porto 0 - 2 Dyn. Kiev [True] 2
2015-11-25 - 20-00 CSKA Moscow 0 - 2 Wolfsburg [] 1
2015-11-25 - 20-00 CSKA Moscow 0 - 2 Wolfsburg [True, True] 2
2015-11-25 - 22-45 Atl. Madrid 2 - 0 Galatasaray [True] 1
2015-11-25 - 22-45 Atl. Madrid 2 - 0 Galatasaray [True] 2
2015-11-25 - 22-45 B. Monchengladbach 4 - 2 Sevilla [] 1
2015-11-25 - 22-45 B. Monchengladbach 4 - 2 Sevilla [True, True, True, True, True, True] 2
2015-11-25 - 22-45 Juventus 1 - 0 Manchester City [True] 1
2015-11-25 - 22-45 Juventus 1 - 0 Manchester City [] 2
2015-11-25 - 22-45 Malmo FF 0 - 5 Paris SG [False, True] 1
2015-11-25 - 22-45 Malmo FF 0 - 5 Paris SG [False, False, True] 2
2015-11-25 - 22-45 Shakhtar Donetsk 3 - 4 Real Madrid [True]

0.6579476861167002